<a href="https://colab.research.google.com/github/nick-kann/PokeForesight/blob/main/PkmnExploratoryAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import requests
import sqlite3
import json
import pandas as pd

# **Creating the Dataset**:

The focus will be on games in the Gen 1 OU format. Only games from higher elo (>=1300) will be included, as higher elo players typically use logic when selecting their leading Pokémon. In contrast, lower elo players often choose the same Pokémon repeatedly or pick randomly, which complicates the model’s learning process. Data will be obtained by making HTTP GET requests to the Pokémon Showdown server.

In [84]:
num_gens = 9
best_gen = 0 # The best generation is the one with the most data
highest_page_count = 0

for gen in range(1, num_gens + 1):
    url = f"https://replay.pokemonshowdown.com/search.json?format=gen{gen}ou&page=100"
    response = requests.get(url)
    data = response.json()
    if not data:
        print(gen)


In [92]:
url = "https://replay.pokemonshowdown.com/search.json?format=gen1ou&sort=rating"
response = requests.get(url)
data = response.json()
data[0]['rating']-10

1626

In [98]:
# Make a HTTP GET request to the Pokemon Showdown server to get the 51 highest rated games for Gen 1 OU
base_url = "https://replay.pokemonshowdown.com/search.json?format=gen1ou&sort=rating"

all_data = []

# Loop from page 1 to 100 since Pokemon Showdown does not show replay records after page 100
for page in range(1, 101):
    url = f"{base_url}&page={page}"
    response = requests.get(url)
    data = response.json()
    data = data[:-1] # Remove the last entry because it matches the first entry of the next page

    if data[0]['rating'] < 1300: # Break if the rating is below 1300
        break

    all_data.extend(data)

In [99]:
df = pd.DataFrame(all_data)
df

,uploadtime,id,format,players,rating,private,password
0,1587300900,gen1ou-1099436824,[Gen 1] OU,"[Eighty Lewis, nsh526625]",1636,0,None
1,1513251517,gen1ou-673803426,[Gen 1] OU,"[RBY e b 0 l a, Roudolf13]",1633,0,None
2,1598815159,gen1ou-1178210825,[Gen 1] OU,"[mi caco addosso, alpha male psyduck]",1632,0,None
3,1472135361,gen1ou-424057776,[Gen 1] OU,"[marcoasd, Peasounay]",1629,0,None
4,1505275274,gen1ou-630923486,[Gen 1] OU,"[RevivalMaya, cholaski]",1610,0,None
...,...,...,...,...,...,...,...
4995,1710180673,gen1ou-2078659415,[Gen 1] OU,"[rbyza cyberacc, RBYZA Skill Issue]",1317,0,None
4996,1709944179,gen1ou-2076763931,[Gen 1] OU,"[minisquidles, Kupcha]",1317,0,None
4997,1705725288,gen1ou-2039515111,[Gen 1] OU,"[Wick_345, TailorRedJacket]",1317,0,None
4998,1703290860,gen1ou-2017910160,[Gen 1] OU,"[DeepestButton, swordofthespirit]",1317,0,None


**With 5000 high-elo replays collected, the next step is to obtain the specific game-data for each replay.**